# Eigensystems

This file will discuss the calculation of eigenvalues for symmetric positive-definite matrices (matrices with positive eigenvalues that eaqual their transpose).


In [1]:
import numpy as np
from numpy import linalg as la

For this notebook, the following 4x4 

In [ ]:
A = np.array([[1.935052353404, 0.887037260957, 1.534235483277, 1.370691224125],
       [0.887037260957, 0.81275989158 , 0.610238622918, 0.999150521052],
       [1.534235483277, 0.610238622918, 1.562260333538, 1.007843062517],
       [1.370691224125, 0.999150521052, 1.007843062517, 1.813258814483]])

## Power method
This is a method to calculate the maximum possible eigenvalue.

This utilises the fact that any vector, $x$,of dimension n can be made up of the eigenvectors, $e_i$ of some nxn matrix, $B$, with eigenvalues such that $|\lambda_1|>|\lambda_2|>...>|\lambda_n|$.
This implies that 
$$ x_o = \alpha_1 e_1 + ... + \alpha_n e_n$$
The non-degenerate case is used, $\lambda_i = \lambda_j$ iff $i = j$.

If $x_o$ is operated on $B$, we get that
$$x_1 = Bx_o = \lambda_1\alpha_1 e_1 + ... + \lambda_n\alpha_n e_n$$
If this operation is repeated $k$ times, we get that 
$$x_k = Bx_{k-1} = {\lambda_1}^k\alpha_1 e_1 + ... + {\lambda_n}^k\alpha_n e_n \approx {\lambda_{1}}^k\alpha_1e_1$$
where $\lambda_1$ is the maximum eigenvalue and k is extremely large. 
We ensure that this does not diverge by having that |$\lambda_1$|>1.

In [2]:
def Powermethod(A,err):
    (m,n) = A.shape
    if(m != n):
        print("Matrix must be square")
        return
    
    # Create a random initial vector
    x = np.random.rand(m)
    
    lam = 0.1
    lamprev = 1
    while np.abs(1-lam/lamprev) > err:
        Ax      = A@x
        lamprev = lam
        lam = la.norm(Ax,2)/la.norm(x,2)
        x = Ax
        
    x = x/la.norm(x)
    return (lam,x)

## Rayleigh-Quotient method
This is an adjusted power method with a higher convergence rate making it take less calculations and therefore time.

It uses the Rayleigh quotient 
$$x_{k+1} = \frac{(x_k)^TBx_k}{(x_k)^Tx_k}$$

where $x_k = Bx_{k-1}$. This gives a denominator that is the dot product of $x_k$ with itself giving the square of the L$_2$-norm of $x_k$.

In [3]:
# Not quite right
def RayleighQuotient(A,err):
    (m,n) = A.shape
    if(m != n):
        print("Matrix must be square")
        return
    
    # Create a random initial vector
    x = np.random.rand(m)
    
    lam = 0.1
    lamprev = 1
    while np.abs(1-lam/lamprev) > err:
        Ax      = A@x
        lamprev = lam
        lam = (np.transpose(x)@(Ax))/(np.transpose(x)@x)
        x = Ax
        
    x = x/la.norm(x)
    return (lam,x)

In [4]:
print(RayleighQuotient(A,1e-10))

NameError: name 'A' is not defined

## Minimum eigenvalue
The minimum eigenvalue can be found using any method that calculates the maximum eigenvalue by using the inverse matrix. 

This is because the maximum eigenvalue of a matrix $A$ is equal to the inverse of the minimum eigenvalue of the matrix $A^{-1}$ and the same is true by simple observation.

## Hotelling's Deflation
This is used to calculate the the second largest eigenvalue for a matrix $A$. This method only works for symmetric matrices.

In this algorthm, the following matrix is constructed

$$ B = A - \lambda_1 e_1\otimes e_1 = A - \lambda_1 e_1{e_1}^T$$
where $\lambda_1$ is the largest eigenvalue, $e_1$ is the corresponding (unit normalized) eigenvector, and $\otimes$ is the outer product.

The matrix $B$ has the same eigenvectors as $A$, and the same eigenvalues except the largest one has been replaced by 0. Thus if we use the power method to find the largest eigenvalue of $B$, this will be the second largest eigenvalue of $A$.
Thus all proceeding eigenvalues can be computed by repeating this.

## Example 
Without using NumPy's la.eig() (or similar functions), find the 4 eigenvalues and eigenvectors of the following matrix. 

In [ ]:
# Finding the maximum eigenvalue from the power method
lmax = Powermethod(A, 1e-10)[0]
xmax = Powermethod(A, 1e-10)[1]
print('Maximum eigenvalue:        ',lmax,'  Corresponding eigenvector:',xmax)

# Finding the second largest eigenvalue from Hotelling's deflation
B = A-lmax*np.outer(xmax,xmax)
l2 = Powermethod(B, 1e-10)[0]
x2 = Powermethod(B, 1e-10)[1]
print('Second largerst eigenvalue:',l2,' Corresponding eigenvector:',x2)

# Finding the third eigenvalue by repeating Hotelling's deflation
C = B-l2*np.outer(x2,x2)
l3 = Powermethod(C, 1e-10)[0]
x3 = Powermethod(C, 1e-10)[1]
print('Third largest eigenvalue:  ',l3,' Corresponding eigenvector:',x3)

# Finding the minimum eigenvalue from the power method on the inverse of A 
lmin = 1/Powermethod(la.inv(A), 1e-10)[0]
xmin = Powermethod(la.inv(A), 1e-10)[1]

# The minimum value had to be normalised
print('Minimum eigenvalue:        ',lmin,' Corresponding eigenvector:',xmin)

In [ ]:
print('Numpy calculation of eigenvalues in descending order:',la.eig(A)[0])
e = la.eig(A)[1]

print('Numpy calculation of eigenvectors as column vectors:\n',la.eig(A)[1])